## Set-up New Conda Environment

conda create -n metabolomics
conda install -c r rpy2 r-essentials r-igraph
conda install -y -c conda-forge jupyter_contrib_nbextensions
conda install -c conda-forge cycler matplotlib biopython

## Import Packages

In [3]:
%matplotlib notebook
import sys, os
#sys.path.insert(0,'/global/homes/b/bpb/repos/metatlas')
sys.path.insert(0, '/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages')

from metatlas.helpers import dill2plots as dp
'''from metatlas.helpers import metatlas_get_data_helper_fun as ma_data
from metatlas.helpers import chromatograms_mp_plots as cp
from metatlas.helpers import chromplotplus as cpp
import metatlas.metatlas_objects as metob
from metatlas.helpers import mzmine_helpers as mzm

import qgrid

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from IPython.display import display

import time
import dill
import numpy as np
import multiprocessing as mp
import pandas as pd

import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)'''

FileNotFoundError: [Errno 2] No such file or directory: '/project/projectdirs/metatlas/mysql_user.txt'

In [ ]:
dp = reload(dp)
# group_names_to_remove = pd.read_csv('/global/homes/b/bpb/Downloads/pks_files_to_remove.csv')['group'].tolist()
# And the groups are:
# 20180104_NEG or POS_12C or 13C 
groups = dp.select_groups_for_analysis(name = '20180104_%_1%C',
                                       most_recent = True,
                                       remove_empty = True,
                                       include_list = [], exclude_list = ['NEG'])#['QC','Blank'])


In [ ]:
for g in groups:
    for f in g.items:
        print(f.name)

In [ ]:
# 20180131_TS_DB_fullcyc_12C_refined_POS
# 20180131_TS_DB_fullcyc_12C_refined_NEG

my_atlas = metob.retrieve('Atlas',name='20180131_TS_DB_fullcyc_12C_refined_POS',username='*')[-1]
my_atlas.name

In [ ]:
from metatlas.helpers import spectralprocessing as spec
new_compound_ids = []
for cid in my_atlas.compound_identifications:
    k,v = spec.parse_formula(cid.compound[0].formula)
    formula = dict(zip(k,v))
    for i in range(1,formula['C']+1):
        temp_mz = cid.mz_references[0].clone()
        temp_mz.mz = cid.mz_references[0].mz + (1.003355 * i)
        new_cid = cid.clone()
        new_cid.mz_references[0] = temp_mz
        new_compound_ids.append(new_cid)
my_atlas.compound_identifications.extend(new_compound_ids)

In [ ]:
atlas_df = ma_data.make_atlas_df(my_atlas)
atlas_df['label'] = [cid.name for cid in my_atlas.compound_identifications]
print my_atlas.name
print my_atlas.username

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)
print(atlas_df.shape)
atlas_df.head(8)

# 6. Get Data

### Much faster data getting script

In [ ]:
all_files = []
for my_group in groups:
    for my_file in my_group.items:
        all_files.append((my_file,my_group,atlas_df,my_atlas))
        
pool = mp.Pool(processes=min(32, len(all_files)))
# from metatlas.helpers.metatlas_get_data_helper_fun import get_data_for_atlas_df_and_file
t0 = time.time()
metatlas_dataset = pool.map(ma_data.get_data_for_atlas_df_and_file, all_files)
pool.close()
pool.terminate()
#If you're code crashes here, make sure to terminate any processes left open.
print time.time() - t0


In [ ]:
dp = reload(dp)
%matplotlib notebook
a = dp.adjust_rt_for_selected_compound(metatlas_dataset,compound_idx=0,include_lcmsruns = [],alpha=0.75,width=10,height=6)

In [ ]:
output_dir = '/global/homes/b/bpb/Downloads/isotope_tests_pos_v2/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

* ### Export Atlas to a Spreadsheet

In [ ]:
atlas_identifications = dp.export_atlas_to_spreadsheet(my_atlas,os.path.join(output_dir,'atlas_export.csv'))

* ### Dataframes and spreadsheets

In [ ]:
dp = reload(dp)
ma_data = reload(ma_data)
peak_height = dp.make_output_dataframe(input_fname = '',input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='peak_height' , output_loc=os.path.join(output_dir,'sheets'))
peak_area = dp.make_output_dataframe(input_fname = my_file,input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='peak_area' , output_loc=os.path.join(output_dir,'sheets'))
mz_peak = dp.make_output_dataframe(input_fname = my_file,input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='mz_peak' , output_loc=os.path.join(output_dir,'sheets'))
rt_peak = dp.make_output_dataframe(input_fname = my_file, input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [],fieldname='rt_peak' , output_loc=os.path.join(output_dir,'sheets'))
mz_centroid = dp.make_output_dataframe(input_fname = my_file,input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='mz_centroid' , output_loc=os.path.join(output_dir,'sheets'))
rt_centroid = dp.make_output_dataframe(input_fname = my_file,input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='rt_centroid' , output_loc=os.path.join(output_dir,'sheets'))

In [ ]:
def organize_isotopic_data(metatlas_dataset):
    """
    takes in a metabolite atlas dataset and returns
    a dictionary where the first keys are unique files
    and the second layer of keys are the unique compound
    names
    """
    
    # make a simplified representation of the names that is the 
    # <name>_<retention time>
    compound_names = ['%s_%s'%(s.split('_')[1],s.split('_')[-1]) for s in ma_data.get_compound_names(metatlas_dataset)[0]]
    # unique_compound_names = pd.unique(compound_names)
    file_names = ma_data.get_file_names(metatlas_dataset)

    # initialize an empty dict of lists
    # first dimension is number of files
    # second dimension is number of compounds
    df = {}
    for i,d1 in enumerate(metatlas_dataset): #iterate through files
        df[file_names[i]] = {}
        for j,d2 in enumerate(d1): #iterate through compounds
            # for each unique compound and file initialize an empty list
            df[file_names[i]][compound_names[j]] = {'data':[],'rt_reference':None}

    for i,d1 in enumerate(metatlas_dataset): #iterate through files
        for j,d2 in enumerate(d1): #iterate through compounds
            # for each unique compound and file append the eic datasets
            df[file_names[i]][compound_names[j]]['data'].append(metatlas_dataset[i][j]['data']['eic'])
            df[file_names[i]][compound_names[j]]['rt_reference'] = metatlas_dataset[i][j]['identification'].rt_references[0]

    return df

isotope_data = organize_isotopic_data(metatlas_dataset)

In [ ]:
f = isotope_data.keys()[0]
c = isotope_data[f].keys()[0]
# isotope_data[f][c]

In [ ]:
files = isotope_data.keys()
compounds = isotope_data[files[0]].keys()
unique_compounds = pd.unique(compounds)
def plot_isotope_eics(isotope_data,my_file,my_compound,first_file_piece=0,last_file_piece=-1,ax=None):
    """
    
    """
    if ax is None:
        ax = plt.gca()
        
    for d in isotope_data[my_file][my_compound]['data']:
        try:
            ax.semilogy(d['rt'],d['intensity'],'-',label='%.4f'%d['mz'][0])
        except:
            #no data
            pass
    rt_info = isotope_data[my_file][my_compound]['rt_reference']
    ax.axvline(rt_info.rt_min,color='black',linewidth=6,alpha=0.5)
    ax.axvline(rt_info.rt_max,color='black',linewidth=6,alpha=0.5)
    ax.axvline(rt_info.rt_peak,color='red',linewidth=6,alpha=0.5)
    shortened_filename = '_'.join(my_file.split('_')[first_file_piece:last_file_piece])
    ax.set_title('%s\n%s'%(shortened_filename,my_compound))
    ax.set_xlabel('Time (min)')
    ax.set_ylabel('Intensity (au)\nLog Scale')
    ax.legend()
    
plt.ioff()

output_dir = '/global/homes/b/bpb/Downloads/isotope_tests_pos_v2/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

alphabet_idx = np.argsort(['_'.join(f.split('_')[7:10]) for f in files])

for cpd in unique_compounds:
    output_filename = os.path.join(output_dir,'%s.pdf'%cpd)
    print(output_filename)
#     if not os.path.isfile(output_filename):
    fig, all_axes = plt.subplots(8, 14, sharey=True, figsize=(80,40))
    all_axes = all_axes.reshape(-1)
    for plot_idx,file_idx in enumerate(alphabet_idx):
        plot_isotope_eics(isotope_data,files[file_idx],cpd,first_file_piece=6,last_file_piece=10,ax=all_axes[plot_idx])
    plt.tight_layout()
    fig.savefig(os.path.join(output_dir,'%s.pdf'%cpd))
    plt.close()

In [ ]:
plt.close()

In [ ]:
len(isotopes)

In [ ]:
def make_isotope_pattern_dataframe(peak_height,rt_peak):
    df = peak_height.copy()
    rt_df = rt_peak.copy()
    rt_df.reset_index(inplace=True,drop=True)
    df.columns = df.columns.droplevel(0)
    rt_df.columns = rt_df.columns.droplevel(0)
    df.fillna(0,inplace=True)
    groups = df.columns.unique()
    for i,row in df.iterrows():
        df.loc[i,'mz'] = i.split('_')[-2]
        df.loc[i,'name'] = i.split('_')[1]
        df.loc[i,'rt'] = i.split('_')[-1]
    df.reset_index(inplace=True,drop=True)

    unique_cpds = df.apply(lambda x: '%s_%s'%(x['name'],x['rt']),axis=1).unique()
    small_dataframes = []
    for cpd in unique_cpds:
        name,rt = cpd.split('_')
        idx = (df['name'] == name) & (df['rt'] == rt)
        small_df = df[idx].copy()
        small_df.drop(['name','rt'],axis=1,inplace=True)
        small_df.set_index(['mz'],inplace=True)
        small_dataframes.append({'compound':cpd,'peak_height':small_df,'rt_peak':rt_df[idx]})
    return small_dataframes
isotopes = make_isotope_pattern_dataframe(peak_height,rt_peak)

In [ ]:
import pickle
with open('/global/homes/b/bpb/Downloads/positive_mode_isotope_data.pkl','w') as fid:
    pickle.dump(isotopes, fid)


In [ ]:
from rdkit import Chem
mol = Chem.MolFromInchi(str(atlas_df.loc[0,'inchi']))
Chem.rdMolDescriptors.CalcMolFormula(mol)

In [ ]:
atlas_df.head(61)

In [ ]:
isotopes[29]['peak_height']

In [ ]:
# len(isotopes)

# cpd = 30
# print(isotopes[cpd]['compound'])

# atlas_df.loc[cpd,:]

# [c.split('_')[7:11] for c in isotopes[cpd]['peak_height'].columns]

# isotopes[cpd]['peak_height'][c].values
# %matplotlib notebook
# cpd = 34
# import matplotlib.pyplot as plt
# fig = plt.figure()
# for c in isotopes[cpd]['peak_height'].columns:
#     if 'Cel' in c:
#         print(c,c.split('_')[9])
#         x = [float(p.replace('p','.')) for p in isotopes[cpd]['peak_height'][c].index]
#         y = isotopes[cpd]['peak_height'][c].values
#         plt.plot(x,y,label=c.split('_')[9])
#         plt.title(isotopes[cpd]['compound'])
# plt.legend()
# plt.tight_layout()

# cpd = 34
# import matplotlib.pyplot as plt
# fig = plt.figure()
# for c in isotopes[cpd]['rt_peak'].columns:
#     if 'Cel' in c:
#         print(c,c.split('_')[9])
#         x = [float(p) for p in isotopes[cpd]['rt_peak'][c].index]
#         y = isotopes[cpd]['rt_peak'][c].values
#         plt.plot(x,y,'o')#,label=c.split('_')[9])
#         plt.title(isotopes[cpd]['compound'])
# plt.legend()
# plt.tight_layout()

# isotopes[cpd]['peak_height'][c for c in isotopes[cpd]['peak_height'].columns if '13C_Cel' in c]

# isotopes[cpd]['rt_peak']

formula = my_atlas.compound_identifications[cpd].compound[0].formula
resolution = 0.3
nat_elements, nat_counts = spec.parse_formula(formula)
nat_isotope_matrix = spec.make_isotope_matrix(nat_elements)#, isotope_dict=element_dict)
nat_dist = spec.make_isotope_distribution(nat_counts, resolution=resolution, *nat_isotope_matrix)[0]
nat_dist[1]*590000

# #TODO: function that takes in a run and a compound and makes nice looking spectra and chroamtograms for each isotopologue

In [ ]:
# df = peak_height.copy()
# df.columns = df.columns.droplevel(1)
# #filter numbers less than threshold with zero
# df.fillna(0,inplace=True)
# #only do calculation for non-zero values
# df_stats = df.transpose().groupby('group').mean().transpose()
# # df_stats = df.transpose().groupby('group').std().transpose()
# # df_stats = df.transpose().groupby('group').count().transpose()

# df_stats.reset_index(drop=True,inplace=True)
# df_stats['label'] = [cid.name for cid in myAtlas.compound_identifications]
# df_stats.to_csv('stats.csv')

# df_stats = df.transpose().groupby('group').mean().transpose()
# df_stats.reset_index(drop=True,inplace=True)
# df_stats['fold_change'] = np.log2(np.divide(df_stats[groups[1]],df_stats[groups[0]]))
# # print df_stats[df_stats.fold_change>-2].index.tolist()
# from scipy.stats import ttest_ind

# cat1 = df[[c for c in df.columns if c == groups[0]]]
# cat2 = df[[c for c in df.columns if c == groups[1]]]


# df_stats['t-score'],df_stats['p-value'] = ttest_ind(cat1,cat2,axis=1)
# df_stats['label'] = [cid.name for cid in myAtlas.compound_identifications]

# df_stats.to_csv('/global/homes/b/bpb/Downloads/magi_coelicolor_stats.csv')

# print(df_stats[df_stats.fold_change<-1].index.tolist())